# Weekly exercise 4: Optimal choice in the bungle good market

In this exercise I ask you to apply the machinery of bungled goods
we have created in video 08 for answering some economic questions.
Make sure you use the code written in that video which is stored
in the notebook *08_bundles_ex2*.

In [1]:
class bundle_good():
    '''Class of bundled goods with well defined arithmetics'''

    items = ('Opera A', 'Opera B', \
             'Ballet A', 'Ballet B', \
             'Symphonic orchestra concert', \
             'Rock opera', \
             'Operetta') # 7 different goods

    def __init__(self,quantities=[0,],price=0.0):
        '''Creates the bundle good object
        '''
        n = len(bundle_good.items) # number of available items
        if len(quantities)<n:
            # add zeros for the unspecified items
            quantities += [0,]*(n-len(quantities))
        elif len(quantities)>n:
            # ignore extra numbers
            quantities = quantities[0:n]
        # create public attributes
        # ensure the quantities in the object are integer
        self.quantities=[int(x) for x in quantities]
        self.price=price

    def __repr__(self):
        '''String representation of the object
        '''
        return 'Bundle object %r with price %1.2f' % (self.quantities,self.price)

    def __add__(self,other):
        '''Addition for bundles: add items and sum prices, or increase price
        '''
        if type(other) is bundle_good:
            # add the quantities using list comprehension with one-to-one matching (zip)
            q1 = [x+y for x,y in zip(self.quantities, other.quantities)]
            # sum of the prices
            p1 = self.price + other.price
            # return new bundle
            return bundle_good(quantities=q1,price=p1)

        elif type(other) in (float,int):
            # increase the price
            p1 = self.price + other
            # return new bundle
            return bundle_good(quantities=self.quantities,price=p1)

        else:
            raise TypeError('Can only add bundle to bundle, or number to bundle price')

    def __sub__(self,other):
        '''Subtraction for bundles: subtract items and prices, or decrease price
        '''
        if type(other) is bundle_good:
            # subtract the quantities using list comprehension with one-to-one matching (zip)
            q1 = [x-y for x,y in zip(self.quantities, other.quantities)]
            # sum of the prices
            p1 = self.price - other.price
            # return new bundle
            return bundle_good(quantities=q1,price=p1)
        elif type(other) in (float,int):
            # decrease the price
            p1 = self.price - other
            # return new bundle
            return bundle_good(quantities=self.quantities,price=p1)
        else:
            raise TypeError('Can only subtract bundle from bundle, or number from bundle price')

    def __mul__(self,num):
        '''Multiplication for bundles: repetition of the original bundle
        '''
        if type(num) is int:
            # multiply quantities using list comprehension
            q1 = [x * num for x in self.quantities]
            # multiply the price
            p1 = self.price * num
            # return new bundle
            return bundle_good(price=p1,quantities=q1)
        else:
            raise TypeError('Can only multiply bundle by an integer')

    def __truediv__(self,num):
        '''Division for bundles: fraction of the original bundle, only if quantities are devisable
        '''
        if type(num) is int:
            # divide quantities and check for divisibility
            q1 = [q//num for q in self.quantities]
            if not all(q%num==0 for q in self.quantities):
                # if can not be devided without a remainder, raise ValueError
                raise ValueError('Can not divide bundle into fractional parts')
            # divide the price
            p1=self.price / num
            # return new bundle
            return bundle_good(price=p1,quantities=q1)
        else:
            raise TypeError('Can only divide bundle by an integer')
            
            

## Optimal choice of bundle goods

Consider a consumer with a utility function over the individual goods
given by

$$
u(x_1,\dots,x_7)=\log(x_1+1)+\big((x_2)^{0.4}+0.5(x_3)^{0.4}\big)^{2.5}-0.5\log(x_4+1)-0.2(x_5*x_6)^{0.2}+2\log(x_7+1).
$$

Find the optimal set of bundle goods to be consumed by comparing
different combinations of the available bundles shown below.

There are only three bundle goods on the market, so we
can afford a brute force optimization algorithm implemented as a
triple nested loop, with each level corresponding to one bundle good
and looping from 0 to some reasonable number (think which number
would be reasonable).

Compute the optimal choice for budgets of 100, 200 and 300 price units.

Use the starter code below.  Each occurrence of **@@@** has to be replaced with appropriate code.

In [5]:
# Available bundle goods
a = bundle_good([2,0,1,3,1,1,0],10.50)
b = bundle_good([0,5,0,4,2,2,2],15.36)
c = bundle_good([1,0,1,2,0,5,4],12.72)
market = [a,b,c]

import math

# utility function
def u(x):
    '''Returns the utility of a bundle'''
    x_1 = x.quantities[0]
    x_2 = x.quantities[1]
    x_3 = x.quantities[2]
    x_4 = x.quantities[3]
    x_5 = x.quantities[4]
    x_6 = x.quantities[5]
    x_7 = x.quantities[6]
    return math.log(x_1 + 1) + (((x_2)**0.4) + ((0.5)*(x_3)**0.4)**2.5) - (math.log(x_4 + 1)*0.5) - (0.2*((x_5 * x_6)**0.2)) + 2*math.log(x_7 + 1) # ok this is disgusting

# optimization routine
def optim(budget,util,market):
    '''Returns the optimal combination of goods at the market, given the budget'''
    nn = len(market) #heuristic for the maximum quantity to check
    # loop over all combination of three bundles to find max utility
    m = -float('inf') #initialize with negative infinity
    for i in range(nn):
        for j in range(nn):
            for k in range(nn):
                bnd = market[i] + market[j] + market[k]  # combination of so many of each of the three bundled goods
                u = util(bnd)
                if bnd.price <= budget and m<u:
                    # found new maximum which is within the budget
                    maxUtil = u  # remember the new max utility
                    optimalComb = bnd  # remember the combination of bundle goods where max is attained
    return optimalComb #return the optimal combination of goods

def report(budget,util,x,market):
    '''Makes nice readable output'''
    return "for budget", budget, "in market", market, "the optimal combination is", x

# main program
for budget in [100, 200, 300]:
    x=optim(budget,u,market)
    ans = report(budget,u,x,market)
    print(ans)

('for budget', 100, 'in market', [Bundle object [2, 0, 1, 3, 1, 1, 0] with price 10.50, Bundle object [0, 5, 0, 4, 2, 2, 2] with price 15.36, Bundle object [1, 0, 1, 2, 0, 5, 4] with price 12.72], 'the optimal combination is', Bundle object [3, 0, 3, 6, 0, 15, 12] with price 38.16)
('for budget', 200, 'in market', [Bundle object [2, 0, 1, 3, 1, 1, 0] with price 10.50, Bundle object [0, 5, 0, 4, 2, 2, 2] with price 15.36, Bundle object [1, 0, 1, 2, 0, 5, 4] with price 12.72], 'the optimal combination is', Bundle object [3, 0, 3, 6, 0, 15, 12] with price 38.16)
('for budget', 300, 'in market', [Bundle object [2, 0, 1, 3, 1, 1, 0] with price 10.50, Bundle object [0, 5, 0, 4, 2, 2, 2] with price 15.36, Bundle object [1, 0, 1, 2, 0, 5, 4] with price 12.72], 'the optimal combination is', Bundle object [3, 0, 3, 6, 0, 15, 12] with price 38.16)


## Sales tax

Imagine the newly introduced sales tax raised all prices by 15%.
Recompute the optimal choices in previous question.

Do the optimal choices change, and how?

In [ ]:
# Write your code here

## Sale for one good

Imagine the sale for bundle good **c** lowers its price by 20%. Recompute
the optimal choices in previous question.

Do the optimal choices change, and how?

In [ ]:
# Write your code here